# Lab-10-2 CNN - MNIST

## Import

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random

## cuda

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

## Parameters

In [3]:
learning_rate = 1e-3
n_epochs = 3
batch_size = 100

## Data Loader

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=False)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=False)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                          batch_size=batch_size, 
                                          shuffle=True, 
                                          drop_last=True)

## Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(2)
        )
    
        self.fc = nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = CNN().to(device)

## Loss Function and Optimizer

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train

In [9]:
total_batch = len(data_loader)
print('>>> Learning Start <<<')

for epoch in range(n_epochs):
    avg_loss = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, Y)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / total_batch
    
    print("[Epoch {}] Loss: {}".format(epoch+1, avg_loss))

print('>>> Learning Finished <<<')

>>> Learning Start <<<
[Epoch 1] Loss: 0.22393347322940826
[Epoch 2] Loss: 0.06211426109075546
[Epoch 3] Loss: 0.04485339671373367
>>> Learning Finished <<<


## Test

In [11]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    Y_pred = model(X_test)
    correct_pred = torch.argmax(Y_pred, 1) == Y_test
    accuracy = correct_pred.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9772999882698059
